# Introduction

Search engines produce two types of clicks for businesses:
* Organic clicks which occur when a user clicks on a normal search result (i.e. not a search ad)
* Paid clicks which occur when a user clicks on an advertisement.

One concern about search engine advertising is that paid clicks will "cannibalize" organic clicks. In other words, if a brand runs a search advertising campaign users might click on the search ad when they otherwise would have clicked on the organic search result.

For example, suppose that a brand averages 500 organic clicks per month.  One month they decide to run a search advertising campaign and this results in 1000 clicks.  But the organic clicks for that month dropped to 400.  Assuming nothing else impacted organic clicks, the "incremental" clicks from the search advertising campaign would be 900 (1000 + 400 - 500). Or, put another way, search advertising cannibalized 100 organic clicks.  If the brand paid \$2.50 CPC for 1,000 clicks, the "cost per incremental click" would the total cost, \$2,500, divided by the number of incremental clicks, 900 = $2.78.

---
A large national retailer is interested in understanding if search engine advertising will impact their organic clicks, and if so, how much.  To estimate this, the retailer chose one of their product categories and ran a search engine advertising campaign for six weeks in 50 designated market areas (DMAs).  DMAs represent metro area across the entire United States - you can read more about them at [Nielsen's website](https://www.nielsen.com/dma-regions/).

The retailer has data for three other product categories that they did NOT run a search advertising campaign for.  The goal of this assignment is to estimate the impact of search advertising on organic clicks.


## Data
There are four product categories:
* Apparel and Accessories
* Hardlines
* Home Furnishings and Decor
* Beauty and Household Essentials

Each product category has several product subcategories.  You can see each product subcategory in one of the data files given below.  Data for organic clicks was collected for a 12 week period starting July 1st, 2023 and ending September 22nd, 2023 for all product categories and product subcategories across all 50 DMAs.  A search advertising campaign was run for keywords relating to "Beauty and Household Essentials" for six weeks starting on August 12th.

There are several data files:
* A mapping of the numeric code for a DMA to the name of the DMA
* A mapping of the product category to the product subcategory.
* Data about the number of organic search clicks for each DMA and each product subcategory. This file contains the following fields:
  * date in YYYY-MM-DD format
  * product subcategory name
  * dma code
  * number of organic clicks
  * number of paid clicks

In [2]:
# Importing Packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Causal Impact Package Import

The code to use Causal Impact is quite simple, and the [Quick Start Guide](https://github.com/google/tfp-causalimpact/blob/main/docs/quickstart.ipynb) is very helpful.

**Note:** The end of the quickstart guide gives tips for working with different date formats.

In [3]:
# TFP CausalImpact

!pip install tfp-causalimpact

In [4]:
import causalimpact

In [5]:
dma_mapping_file = 'https://raw.githubusercontent.com/hangingbelay/spring2024/main/homework_3_dma_mapping.csv'

product_category_mapping_file = 'https://raw.githubusercontent.com/hangingbelay/spring2024/main/homework_3_product_category_mapping.csv'

click_data_file = 'https://raw.githubusercontent.com/hangingbelay/spring2024/main/homework_3_click_data.csv'

In [6]:
# Files
import pandas as pd
import numpy as np

# dma_mapping
dma_mapping_file = pd.read_csv(dma_mapping_file)
print(dma_mapping_file.head())

# product_category_mapping_file
product_category_mapping_file = pd.read_csv(product_category_mapping_file, header = None)
product_category_mapping_file.columns = ['Product_Category', 'Product_Sub_Category']
print('\n')
print(product_category_mapping_file.head())

# click_data_file
click_data_file = pd.read_csv(click_data_file)
print('\n')
print(click_data_file.head())

   code                                           name
0  1001               Albuquerque-Santa Fe, New Mexico
1  1002                               Atlanta, Georgia
2  1003                                  Austin, Texas
3  1004                            Baltimore, Maryland
4  1005  Birmingham (Anniston and Tuscaloosa), Alabama


          Product_Category Product_Sub_Category
0  apparel_and_accessories       womens_apparel
1  apparel_and_accessories         mens_apparel
2  apparel_and_accessories         boys_apparel
3  apparel_and_accessories        girls_apparel
4  apparel_and_accessories     toddlers_apparel


         date product_sub_category  dma_code  organic_clicks  paid_clicks
0  2023-07-01       womens_apparel      1001               2            0
1  2023-07-01       womens_apparel      1002               6            0
2  2023-07-01       womens_apparel      1003               4            0
3  2023-07-01       womens_apparel      1004               7            0
4  2023-

In [7]:
# Table info

print(dma_mapping_file.info())

print('\n')
print(product_category_mapping_file.info())

print('\n')
print(click_data_file.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    50 non-null     int64 
 1   name    50 non-null     object
dtypes: int64(1), object(1)
memory usage: 928.0+ bytes
None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Product_Category      25 non-null     object
 1   Product_Sub_Category  25 non-null     object
dtypes: object(2)
memory usage: 528.0+ bytes
None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105000 entries, 0 to 104999
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   date                  105000 non-null  object
 1   product_sub_category  105000 non-null  object
 2   dma_code           

I used the data files, aggregate them for organic clicks and paid clicks for each product category.  

Before aggregating, I verified that each time series for each product subcategory and dma is "regular" - i.e. there are no missing dates.  

In [8]:
# Checking for null values

click_data_file['date'].where(click_data_file['date'].isna()).count()

0

In [9]:
# Converting to Dict & Mapping column

categoryDict = dict(zip(product_category_mapping_file['Product_Sub_Category'], product_category_mapping_file['Product_Category']))
print(categoryDict)

click_data_file['product_sub_category'] = click_data_file['product_sub_category'].map(categoryDict)

click_data_file = click_data_file.rename(columns = {'product_sub_category': 'product_category'})

{'womens_apparel': 'apparel_and_accessories', 'mens_apparel': 'apparel_and_accessories', 'boys_apparel': 'apparel_and_accessories', 'girls_apparel': 'apparel_and_accessories', 'toddlers_apparel': 'apparel_and_accessories', 'newborn_apparel': 'apparel_and_accessories', 'jewelery': 'apparel_and_accessories', 'accessories': 'apparel_and_accessories', 'shoes': 'apparel_and_accessories', 'video_games': 'hardlines', 'toys': 'hardlines', 'entertainment': 'hardlines', 'sporting_goods': 'hardlines', 'luggage': 'hardlines', 'furniture': 'home_furnishings_and_decor', 'storage': 'home_furnishings_and_decor', 'kitcheware': 'home_furnishings_and_decor', 'bed_and_bath': 'home_furnishings_and_decor', 'home_improvement': 'home_furnishings_and_decor', 'home_decor': 'home_furnishings_and_decor', 'beauty_and_personal_care': 'beauty_and_household_essentials', 'baby_gear': 'beauty_and_household_essentials', 'cleaning': 'beauty_and_household_essentials', 'paper_products': 'beauty_and_household_essentials', '

In [10]:
# Checking for Paid Clicks are only for beauty_and_household_essentials

click_data_file.groupby(['product_category']).sum().drop('date', axis=1)

,dma_code,organic_clicks,paid_clicks
product_category,,,
apparel_and_accessories,38763900,202303,0
beauty_and_household_essentials,21535500,297273,71178
hardlines,21535500,148715,0
home_furnishings_and_decor,25842600,155418,0


We can see that the only product category with paid clicks is 'Beauty and Household Essentials', and the only dates with paid clicks are during the search advertising campaign (8/12/2023 - 9/22/2023)

In [11]:
# Checking for paid_clicks during the search advertising campaign (8/12/2023 - 9/22/2023)

results_df = click_data_file['paid_clicks'].where((click_data_file['date'] >= '2023-08-12') & (click_data_file['date'] <= '2023-09-22')).sum()
results_df

71178.0

In [12]:
# Aggregating Organic and Paid Clicks

clicks_df = click_data_file.groupby(['date', 'product_category']).agg(
    organic_clicks=('organic_clicks', 'sum'),
    paid_clicks=('paid_clicks', 'sum')).reset_index()
clicks_df.head(n=4)

,date,product_category,organic_clicks,paid_clicks
0,2023-07-01,apparel_and_accessories,2643,0
1,2023-07-01,beauty_and_household_essentials,4859,0
2,2023-07-01,hardlines,2117,0
3,2023-07-01,home_furnishings_and_decor,2520,0


Now I will estimate the counterfactual for organic clicks for Beauty and Household Essentials during the search advertising campaign.

To do this, we will consider each of the other three product categories with no search advertising as a possible option for a synthethic control.  

For each option, run a Causal Impact model using the number of organic clicks for Beauty and Household Essentials and the number of organic clicks for the other product category.  

Through this analysis we will get to know of which product category is the best synthethic control for Beauty and Household Essentials.

---

Causal Impact rests on three main assumptions:
* Predictability
* Unaffectedness
* Stability



In [13]:
# Write your code for question #2 here

# Organic clicks for each product_category

app_acc = click_data_file.where(click_data_file['product_category'] == 'apparel_and_accessories')

beat_house_ess = click_data_file.where(click_data_file['product_category'] == 'beauty_and_household_essentials')

hardlines = click_data_file.where(click_data_file['product_category'] == 'hardlines')

home_fur_decor = click_data_file.where(click_data_file['product_category'] == 'home_furnishings_and_decor')


In [14]:
# Organic Clicks Dataframe

organicClick = pd.DataFrame()

organicClick['date'] = click_data_file['date']
organicClick['app_acc'] = app_acc['organic_clicks']
organicClick['beat_house_ess'] = beat_house_ess['organic_clicks']
organicClick['hardlines'] = hardlines['organic_clicks']
organicClick['home_fur_decor'] = home_fur_decor['organic_clicks']

organicClick = organicClick.reset_index()
organicClick['date'] = pd.to_datetime(organicClick['date'])
organicClick = organicClick.drop(['index'], axis=1).groupby('date').sum()

organicClick

,app_acc,beat_house_ess,hardlines,home_fur_decor
date,,,,
2023-07-01,2643.0,4859.0,2117.0,2520.0
2023-07-02,1490.0,2947.0,1760.0,1376.0
2023-07-03,1567.0,2825.0,1399.0,1345.0
2023-07-04,1726.0,3244.0,1337.0,1574.0
2023-07-05,1986.0,3663.0,1800.0,1892.0
...,...,...,...,...
2023-09-18,1657.0,2516.0,1981.0,1378.0
2023-09-19,1680.0,2905.0,1458.0,1606.0
2023-09-20,2073.0,3398.0,1949.0,1905.0


In [15]:
# Causality Model

pre = ('2023-07-01', '2023-08-11')
post = ('2023-08-12', '2023-09-22')

app_acc_causal = causalimpact.fit_causalimpact(data=organicClick[["beat_house_ess","app_acc"]], pre_period=pre , post_period=post)
hardlines_causal = causalimpact.fit_causalimpact(data=organicClick[["beat_house_ess","hardlines"]], pre_period=pre , post_period=post)
home_fur_decor_causal = causalimpact.fit_causalimpact(data=organicClick[["beat_house_ess","home_fur_decor"]], pre_period=pre , post_period=post)

In [16]:
# apparel_and_accesories

causalimpact.plot(app_acc_causal)

alt.FacetChart(...)

In [17]:
# harlines

causalimpact.plot(hardlines_causal)

alt.FacetChart(...)

In [18]:
# home_furnishings_and_decor

causalimpact.plot(home_fur_decor_causal)

alt.FacetChart(...)

Based on the graphs, I find **'Home Furniture Decor'** to be the best synthetic control for 'Beauty and Household Essentials'. The graph appears to be the best among the others because the observed data closely follows the predicted mean before the intervention, indicating a stable and predictable control. Post-intervention, the pointwise differences are smaller and less volatile compared to previous graphs, and the cumulative effect shows a more gradual trend, suggesting that the observed data remains close to the counterfactual prediction. This behavior implies that the synthetic control has captured the pre-intervention characteristics well, leading to a more reliable estimation of the intervention's impact. The stable confidence intervals before and after the intervention underscore the reliability of **'Home Furniture Decor'** as a synthetic control for **'Beauty and Household Essentials'**.


The first impact model (**apparel_and_accessories**) shows an intervention causing observed values to fall below the predicted mean within a widening confidence interval, signaling a clear causal impact and growing predictive uncertainty post-intervention.

The second impact model (**hardlines**) displays a marked post-intervention deviation of observed values from the predicted, within an expanding confidence interval, denoting a significant negative impact and escalating uncertainty in the model's predictions.

Using the results of the analyses I will be answering the below questions

1. What is the cumulative impact of search advertising on the organic clicks for Beauty and Household Essentials?

**Note:** To get this from the model by we can run something like the following code:


```
# assume your model is named impact
print(causalimpact.summary(impact, output_format='summary'))
```
To calculate it manually we can use the point effects estimates during the period of the advertising campaign.  If your model is named impact, there is a dataframe with this information:


```
# assume your model is named impact.  This gives a dataframe with lots of info.
impact.series
```




2. If the total cost of the search advertising campaign was $150,000, calculate the "cost per incremental click", following the methodology given in the introduction.  What is the relative difference between the normal cost per click, and the cost per incremental click?

In [19]:
import warnings

# Suppress FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Run the causal impact summary
print(causalimpact.summary(home_fur_decor_causal, output_format='summary'))


Posterior Inference {CausalImpact}
                          Average            Cumulative
Actual                    3334.9             140066.0
Prediction (s.d.)         3725.7 (76.43)     156481.4 (3209.93)
95% CI                    [3579.6, 3869.9]   [150343.0, 162535.0]

Absolute effect (s.d.)    -390.8 (76.43)     -16415.4 (3209.93)
95% CI                    [-535.0, -244.7]   [-22469.0, -10277.0]

Relative effect (s.d.)    -10.4% (1.8%)      -10.4% (2.0%)
95% CI                    [-13.8%, -6.8%]    [-13.8%, -6.8%]

Posterior tail-area probability p: 0.001
Posterior prob. of a causal effect: 99.89%

For more details run the command: summary(impact, output_format="report")


In [20]:
print(causalimpact.summary(home_fur_decor_causal, output_format = 'summary'))


Posterior Inference {CausalImpact}
                          Average            Cumulative
Actual                    3334.9             140066.0
Prediction (s.d.)         3725.7 (76.43)     156481.4 (3209.93)
95% CI                    [3579.6, 3869.9]   [150343.0, 162535.0]

Absolute effect (s.d.)    -390.8 (76.43)     -16415.4 (3209.93)
95% CI                    [-535.0, -244.7]   [-22469.0, -10277.0]

Relative effect (s.d.)    -10.4% (1.8%)      -10.4% (2.0%)
95% CI                    [-13.8%, -6.8%]    [-13.8%, -6.8%]

Posterior tail-area probability p: 0.001
Posterior prob. of a causal effect: 99.89%

For more details run the command: summary(impact, output_format="report")


In [21]:
impactData = home_fur_decor_causal.series
impactData.reset_index()
impactData

,observed,posterior_mean,posterior_lower,posterior_upper,point_effects_mean,point_effects_lower,point_effects_upper,cumulative_effects_mean,cumulative_effects_lower,cumulative_effects_upper,pre_period_start,pre_period_end,post_period_start,post_period_end,time
date,,,,,,,,,,,,,,,
2023-07-01,4859.0,5014.930664,4396.414807,5581.800891,-155.930664,-722.800891,462.585193,0.000000,0.000000,0.000000,2023-07-01,2023-08-11,2023-08-12,2023-09-22,2023-07-01
2023-07-02,2947.0,2840.975342,2266.412958,3469.733167,106.024658,-522.733167,680.587042,0.000000,0.000000,0.000000,2023-07-01,2023-08-11,2023-08-12,2023-09-22,2023-07-02
2023-07-03,2825.0,2781.855469,2172.741425,3425.971021,43.144531,-600.971021,652.258575,0.000000,0.000000,0.000000,2023-07-01,2023-08-11,2023-08-12,2023-09-22,2023-07-03
2023-07-04,3244.0,3217.270020,2616.533820,3863.132996,26.729980,-619.132996,627.466180,0.000000,0.000000,0.000000,2023-07-01,2023-08-11,2023-08-12,2023-09-22,2023-07-04
2023-07-05,3663.0,3821.534180,3210.285303,4376.045569,-158.534180,-713.045569,452.714697,0.000000,0.000000,0.000000,2023-07-01,2023-08-11,2023-08-12,2023-09-22,2023-07-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-18,2516.0,2836.345215,2253.617346,3414.477478,-320.345215,-898.477478,262.382654,-15227.897461,-20865.149725,-9785.412378,2023-07-01,2023-08-11,2023-08-12,2023-09-22,2023-09-18
2023-09-19,2905.0,3269.174072,2669.090997,3901.535205,-364.174072,-996.535205,235.909003,-15592.071533,-21356.663904,-9952.884332,2023-07-01,2023-08-11,2023-08-12,2023-09-22,2023-09-19
2023-09-20,3398.0,3837.578613,3230.458911,4415.383240,-439.578613,-1017.383240,167.541089,-16031.650146,-21876.315277,-10214.268817,2023-07-01,2023-08-11,2023-08-12,2023-09-22,2023-09-20


In [22]:
print(causalimpact.summary(home_fur_decor_causal, output_format = 'report'))


Analysis report {CausalImpact}


During the post-intervention period, the response variable had
an average value of approx. 3334.9. By contrast, in the absence of an
intervention, we would have expected an average response of 3725.7.
The 95% interval of this counterfactual prediction is [3579.6, 3869.9].
Subtracting this prediction from the observed response yields
an estimate of the causal effect the intervention had on the
response variable. This effect is -390.8 with a 95% interval of
[-535.0, -244.7]. For a discussion of the significance of this effect,
see below.


Summing up the individual data points during the post-intervention
period (which can only sometimes be meaningfully interpreted), the
response variable had an overall value of 140066.0.
By contrast, had the intervention not taken place, we would have expected
a sum of 156481.4. The 95% interval of this prediction is [150343.0, 162535.0].


The above results are given in terms of absolute numbers. In relative
terms, the

In [23]:
# Normal CPC

normalCPC = round(150000/ clicks_df['paid_clicks'].sum(), 3)

normalCPC

2.107

In [24]:
# Incremental cost per click

postData = impactData.loc[impactData.index >= '2023-08-12']

incCPC = round(150000/ (postData['point_effects_mean'].sum() + clicks_df['paid_clicks'].sum()), 3)
incCPC

2.739

In [25]:
# Causal Effect on Organic Clicks

postData['point_effects_mean'].sum()

-16415.3935546875

In [26]:
((2.738-2.11)/2.11)*100

29.763033175355456

Cannibalization led to a reduction of 16,256 clicks, causing the cost per click (CPC) to rise from \$2.11 to \$2.73, marking an overall CPC increase of 29.76%.

Instead of using other product categories as a synthethic control, another option would be to use a DMA (or multiple DMAs) as a synthethic control.  In other words, the retailer could have chosen some DMAs to run search advertising for, and the other DMAs receive no advertising.

Choose a single DMA to use as a synthethic control.  Create two time series for organic clicks:

* For all product categories with only your chosen DMA
* For all product categories without your chosen DMA.  

I built a causal impact model and analyzed how well the single DMA performs as a synthetic control.

I also justified why I chose this DMA

In [27]:
# Write your code for question #4 here

categoryDict = dict(zip(dma_mapping_file['code'], dma_mapping_file['name']))

click_data_file['dma_code'] = click_data_file['dma_code'].map(categoryDict)

click_data_file = click_data_file.rename(columns = {'dma_code': 'dma'})

click_data_file.head(n=4)

,date,product_category,dma,organic_clicks,paid_clicks
0,2023-07-01,apparel_and_accessories,"Albuquerque-Santa Fe, New Mexico",2,0
1,2023-07-01,apparel_and_accessories,"Atlanta, Georgia",6,0
2,2023-07-01,apparel_and_accessories,"Austin, Texas",4,0
3,2023-07-01,apparel_and_accessories,"Baltimore, Maryland",7,0


In [28]:
click_data_file[click_data_file["date"] >= "2023-08-12"].groupby("dma").agg({"paid_clicks": "sum"}).sort_values(by = "paid_clicks", ascending = False)

,paid_clicks
dma,
"New York, New York",12190
"Los Angeles, California",5645
"Chicago, Illinois",3932
"Philadelphia, Pennsylvania",3317
"Dallas-Fort Worth, Texas",2245
"Washington, D.C. (Hagerstown)",2237
"Houston, Texas",2108
"San Francisco-Oakland-San Jose, California",2052
"Boston (Manchester), Massachusetts",1850


In [29]:
control_category = 'New York, New York'

# Extracting series for treatment and control group

# Time Series without the chosen DMA
treatment = click_data_file[click_data_file['dma'] != control_category]
treatment_agg = treatment.groupby('date')['organic_clicks'].sum()

# Time Series w/ the chosen DMA
control = click_data_file[click_data_file['dma'] == control_category]
control_agg = control.groupby('date')['organic_clicks'].sum()

# Causal Impact Model
combined_data_dma = pd.DataFrame(data={"treatment":np.array(treatment_agg),"control":np.array(control_agg)},index=pd.date_range('2023-07-01','2023-09-22'))

In [30]:
impact_dma = causalimpact.fit_causalimpact(combined_data_dma, pre, post)

causalimpact.plot(impact_dma)

alt.FacetChart(...)

**New York** is the **best DMA** for a causal impact model or synthetic control due to its large data volume for robust statistical analysis, diverse market that represents national trends, and its economic influence, making it a strong proxy for broader patterns.

The causal impact graph illustrates that **New York DMA**, as a synthetic control, accurately mirrors the pre-intervention pattern of organic clicks across all product categories, showcasing its effectiveness in predicting the counterfactual outcome and indicating its suitability as a synthetic control in the causal impact model. The consistent and narrow **confidence intervals** in the data further suggest high precision and reliability in the estimates derived from this DMA.